In [1]:
import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# os.environ["TORCH_LOGS"] = "output_code"
# os.environ["TORCH_LOGS"] = "inductor"
# os.environ["TORCHINDUCTOR_TRACE"] = "1"
# os.environ["TORCHINDUCTOR_VERBOSE"] = "1"
# os.environ["TORCHINDUCTOR_DEBUG"] = "1"
# os.environ["TORCHINDUCTOR_DUMP"] = "1"
# os.environ["TORCHINDUCTOR_COMPILE_THREADS"] = "1"
# os.environ["TORCH_COMPILE_DEBUG"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = (
    # "backend:cudaMallocAsync,"
    "expandable_segments:True,"
    # "garbage_collection_threshold:0.6"
)

import sys
sys.path.append('/home/hice1/yyu496/kaggle/CW')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch._functorch.aot_autograd import aot_module, make_boxed_func
from torch.autograd import grad
from torchvision.datasets import ImageFolder
from torch.optim.lr_scheduler import (
    LinearLR,
    CosineAnnealingLR,
    SequentialLR
)
from torch.optim.optimizer import Optimizer

import torch
from torch.profiler import profile, record_function, ProfilerActivity
import torch._dynamo as dynamo
import torch.nn.utils.parametrize as P
from torch.nn.utils.parametrizations import spectral_norm

import ACT6.cpp_extension as cpp_extension

from ACT6.controller import Controller
import ACT6.cuda_graph_utils as cuda_utils
from ACT6.layers import RMSNorm, DOConv2d, DOConv1d, DOLinear
from ACT6.fusion.fused_layers import DOBatchNormReLU2d

import timm
from timm.layers import trunc_normal_

import torchvision.models as models
from torchvision.transforms import v2

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.allow_tf32 = True

import random
import numpy as np
import copy


import actnn
# available choices are ["L0", "L1", "L2", "L3", "L4", "L5"]
actnn.set_optimization_level("L3")


from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchmetrics import Accuracy 


import matplotlib.pyplot as plt
import math

import triton
import triton.language as tl    

/home/hice1/yyu496/.conda/envs/lib/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ACT6.modules.tensor_act_reshape_utils import (
    spatical_aware_act_tensor_reshape,
    spatical_aware_act_tensor_reshape_back,
    default_act_tensor_reshape,
    default_act_tensor_reshape_back)
    

In [3]:
X = torch.rand(64, 512, 32, 32, device='cuda')
Group_Size = 512


In [69]:
X2, N, G, new_H, new_W, ori_shape_new, spatical_padding_eligibility, spatical_reshape_eligibility,channel_mean = spatical_aware_act_tensor_reshape(X, Group_Size=16, act_padding=False, pack_only=False)

In [70]:
ori_X_2 = spatical_aware_act_tensor_reshape_back(X2,ori_shape_new, Group_Size=16, new_H=new_H, new_W=new_W, act_padding=False, spatical_padding_eligibility=spatical_padding_eligibility, spatical_reshape_eligibility=spatical_reshape_eligibility, channel_mean=channel_mean)

In [71]:
X3, N_old, G_old, original_shape_old = default_act_tensor_reshape(X, Group_Size=16, act_padding=False)

TypeError: default_act_tensor_reshape() missing 1 required positional argument: 'pack_only'

In [72]:
ori_X_3 = default_act_tensor_reshape_back(X3, original_shape_old, act_padding=False)

NameError: name 'X3' is not defined

In [6]:
# x = torch.randn(8, 512, 32, 32, device='cuda')

group_size = 512
N = X.size(0)
X2 = X.view(N, -1, group_size)
G = X2.size(1)

In [14]:
X2 = torch.clamp(X2, -3, 3)

min = X2.min()
max = X2.max()

# mean_X2 = X2.mean()
# var_X2 = X2.var()
# _X2 = (X2 - mean_X2) / torch.sqrt(X2.var() + 1e-6)
# rms_norm = torch.sqrt((X2 ** 2).mean(dim=0, keepdim=True) + 1e-6)
# _X2 = X2 / rms_norm
# _X2 = F.tanh(X2 / 0.5) * 0.5
# _X2 = F.silu(X2)
# _X2 = F.gelu(_X2) 
# _X2 = F.selu(X2)
# _X2 = F.softshrink(X2, lambd=0.7)
# _X2 = F.rrelu(X2)
# _mean_X2 = _X2.mean()
# _var_X2 = _X2.var()
# _X2 = (_X2 - _mean_X2) / torch.sqrt(_var_X2 + 1e-6)
# _X2 = F.gelu(X2)


scale = (max - min) / (2**2 - 1)

_x2_quantized = (_X2 - min) / scale
x2_quantized = (X2 - min) / scale

# X2_hat_quantized = (X2_hat - min) / scale

In [15]:
X2.shape

torch.Size([64, 1024, 512])

In [16]:
_X2.shape

torch.Size([64, 1024, 512])

In [17]:
# X2_hat_quantized_checker = X2_hat_quantized.to(torch.int32).view(N, -1, Group_Size // 4, 4)

In [18]:
# _X2_quantized_checker = _x2_quantized.view(N, -1, Group_Size // 2, 2)
# _X2_quantized_checker = _X2_quantized_checker.mean(dim=-1).repeat(1, 1, 1, 2).to(torch.int32)
_X2_quantized_checker = _x2_quantized.to(torch.int32).view(N, -1, Group_Size // 2, 2)

X2_quantized_checker = x2_quantized.to(torch.int32).view(N, -1, Group_Size // 2, 2)

In [19]:
_X2_dequantized = (_X2_quantized_checker.to(torch.float32)+ min) * scale
X2_dequantized = (X2_quantized_checker.to(torch.float32)+ min) * scale

# F.cosine_similarity(_X2_dequantized.view(-1), X2_dequantized.view(-1), dim=0)
F.pairwise_distance(_X2_dequantized.view(-1), X2_dequantized.view(-1), p=2)

tensor(1308.6737, device='cuda:0')

In [20]:
print(f"_X2 number of 0: {torch.sum(_X2_quantized_checker == 0)}")
print(f"_X2number of 1: {torch.sum(_X2_quantized_checker == 1)}")
print(f"_X2 number of 2: {torch.sum(_X2_quantized_checker == 2)}")
print(f"_X2 number of 3: {torch.sum(_X2_quantized_checker == 3)}")


_X2 number of 0: 17771842
_X2number of 1: 13424074
_X2 number of 2: 2358516
_X2 number of 3: 0


In [21]:
print(f"Equal {torch.sum(_X2_quantized_checker[..., 0] == _X2_quantized_checker[..., 1])}")
print(f"Not Equal {torch.sum(_X2_quantized_checker[..., 0] != _X2_quantized_checker[..., 1])}")
print(f"Ratio: {torch.sum(_X2_quantized_checker[..., 0] != _X2_quantized_checker[..., 1]) / _X2_quantized_checker.numel()}")

Equal 7474254
Not Equal 9302962
Ratio: 0.27724987268447876


In [22]:
print(f"Equal {torch.sum(X2_quantized_checker[..., 0] == X2_quantized_checker[..., 1])}")
print(f"Not Equal {torch.sum(X2_quantized_checker[..., 0] != X2_quantized_checker[..., 1])}")
print(f"Ratio: {torch.sum(X2_quantized_checker[..., 0] != X2_quantized_checker[..., 1]) / X2_quantized_checker.numel()}")

Equal 5590171
Not Equal 11187045
Ratio: 0.3333999216556549


In [23]:
print(f"X2 number of 0: {torch.sum(X2_quantized_checker == 0)}")
print(f"X2 number of 1: {torch.sum(X2_quantized_checker == 1)}")
print(f"X2 number of 2: {torch.sum(X2_quantized_checker == 2)}")
print(f"X2 number of 3: {torch.sum(X2_quantized_checker == 3)}")

X2 number of 0: 11185916
X2 number of 1: 11182192
X2 number of 2: 11186324
X2 number of 3: 0


In [23]:
_X2_dequantized = (_X2_quantized_checker.to(torch.float32)+ min) * scale
X2_dequantized = (X2_quantized_checker.to(torch.float32)+ min) * scale

In [450]:
F.cosine_similarity(_X2_dequantized.view(-1), X2_dequantized.view(-1), dim=0)

tensor(0.5640, device='cuda:0')

In [336]:
x2_quantized_int = x2_quantized.to(torch.int32)
x2_quantized_int.view(N, G, group_size // 4, 4)

tensor([[[[1, 1, 1, 0],
          [0, 1, 2, 1],
          [1, 2, 1, 0],
          ...,
          [2, 1, 1, 2],
          [1, 0, 0, 0],
          [0, 2, 1, 2]],

         [[0, 1, 2, 2],
          [0, 0, 2, 0],
          [2, 0, 1, 1],
          ...,
          [0, 1, 2, 2],
          [0, 1, 1, 1],
          [0, 0, 2, 1]],

         [[1, 2, 1, 2],
          [0, 1, 0, 1],
          [1, 2, 1, 0],
          ...,
          [0, 2, 0, 2],
          [1, 2, 2, 1],
          [0, 0, 2, 0]],

         ...,

         [[2, 2, 1, 0],
          [0, 2, 1, 2],
          [2, 2, 1, 2],
          ...,
          [2, 2, 2, 2],
          [0, 1, 1, 2],
          [0, 2, 1, 0]],

         [[0, 1, 0, 0],
          [0, 1, 2, 2],
          [2, 2, 1, 0],
          ...,
          [1, 2, 2, 0],
          [1, 0, 1, 0],
          [0, 0, 1, 0]],

         [[1, 2, 1, 1],
          [0, 0, 2, 1],
          [2, 1, 0, 0],
          ...,
          [1, 2, 0, 1],
          [0, 2, 1, 1],
          [1, 2, 0, 1]]],


        [[[0, 1, 

In [18]:
x2_quantized.view(N, G, group_size // 2, 2)

tensor([[[[2.3432e+00, 1.0879e+00],
          [4.8966e-02, 1.5319e+00],
          [1.9327e+00, 7.5273e-01],
          ...,
          [2.4140e+00, 6.5493e-01],
          [8.3042e-01, 2.9752e+00],
          [1.3666e+00, 1.0859e+00]],

         [[1.8584e+00, 9.9521e-01],
          [1.6998e-01, 1.5811e+00],
          [2.2233e+00, 1.8326e+00],
          ...,
          [7.9671e-01, 2.5218e+00],
          [1.2742e-01, 9.8014e-01],
          [1.4920e-02, 1.4914e+00]],

         [[2.0787e+00, 7.5452e-01],
          [6.1217e-02, 7.4855e-01],
          [6.2302e-01, 7.5215e-01],
          ...,
          [1.1050e-01, 1.4306e-01],
          [1.5507e+00, 2.8007e+00],
          [7.5011e-01, 1.8909e+00]],

         ...,

         [[1.6356e+00, 1.7314e+00],
          [1.3093e+00, 2.4303e+00],
          [7.4922e-01, 3.1696e-01],
          ...,
          [7.9777e-01, 2.8208e+00],
          [1.0357e+00, 9.8567e-01],
          [2.5519e+00, 1.8857e+00]],

         [[1.1974e+00, 1.8758e+00],
          [8.4026

In [ ]:
def h4(x):
    a,b,c,d = x[...,0],x[...,1],x[...,2],x[...,3]
    return torch.stack([
        a+b+c+d,
        a+b-c-d,
        a-b+c-d,
        a-b-c+d
    ], dim=-1) * 0.5


def h4_inv(y):
    y0,y1,y2,y3 = y[...,0],y[...,1],y[...,2],y[...,3]
    return torch.stack([
        y0+y1+y2+y3,
        y0+y1-y2-y3,
        y0-y1+y2-y3,
        y0-y1-y2+y3
    ], dim=-1) * 0.5


def hadamard_auto(x):
    shp = x.shape
    B = x.shape[0]

    flat = x.reshape(B, -1).contiguous()
    F = flat.shape[1]

    pad = (-F) % 4

    if pad:
        flat = torch.cat(
            [flat, torch.zeros(B, pad, device=x.device, dtype=x.dtype)],
            dim=1
        )

    Fp = flat.shape[1]

    g = flat.reshape(B, Fp // 4, 4)
    y = h4(g)

    # drop padding before reshaping back
    y = y.reshape(B, Fp)
    if pad:
        y = y[:, :-pad]

    return y.reshape(shp), pad




def hadamard_auto_inv(y, pad):
    shp = y.shape
    B = y.shape[0]

    flat = y.reshape(B, -1).contiguous()
    F = flat.shape[1]

    if pad:
        flat = torch.cat(
            [flat, torch.zeros(B, pad, device=y.device, dtype=y.dtype)],
            dim=1
        )

    Fp = flat.shape[1]

    g = flat.reshape(B, Fp // 4, 4)
    x = h4_inv(g)

    x = x.reshape(B, Fp)
    if pad:
        x = x[:, :-pad]

    return x.reshape(shp)






In [29]:
x = torch.randn(64,32,8,8,device='cuda')

y, pad = hadamard_auto(x)
xr = hadamard_auto_inv(y,  pad)

print((x-xr).abs().max())


tensor(4.7684e-07, device='cuda:0')


In [30]:
y.shape

torch.Size([64, 32, 8, 8])

In [4]:
import cupy as cp

In [7]:
x = torch.randn(8, 512, 32, 32, device='cuda')
x2 = x.permute(1, 0, 2, 3).contiguous()

In [10]:
x2_torch = x2.reshape(512, -1)
x2_cupy = cp.from_dlpack(torch.utils.dlpack.to_dlpack(x2_torch))

In [16]:
x2_cupy.shape

(512, 8192)

In [12]:
x_res = cp.dot(x2_cupy, x2_cupy.T)

In [13]:
k = 8

U, S, _ = cp.linalg.svd(x_res, full_matrices=False)
P = U[:, :k]


In [17]:
low_rank_x2 = cp.dot(P.T, x2_cupy)

In [21]:
low_rank_x2_new = low_rank_x2.reshape(8, 8, 32, 32).transpose(1, 0, 2, 3)

In [25]:
low_rank_x2_new_torch = torch.utils.dlpack.from_dlpack(low_rank_x2_new)

In [27]:
low_rank_x2_new_torch.shape

torch.Size([8, 8, 32, 32])

In [35]:
U, S, _ = cp.linalg.svd(x2_cupy, full_matrices=False)

In [36]:
energy = (S**2)
ratio = cp.cumsum(energy) / cp.sum(energy)

In [44]:
k = cp.searchsorted(ratio, cp.array(0.10))

In [45]:
k

array(34)

In [48]:
B, C, H, W = x.shape

In [50]:
new = U[:, :k].reshape(int(k), B, H, W)

ValueError: cannot reshape array of size 17408 into shape (34, 8, 32, 32)